<img style='margin-right:0' src="http://dinfo.ca/logoDptInfo.jpg" width=300>

# Référence Fichiers Binaires (Python)
---

In [3]:
print('%x' % ord('9'))

39


In [4]:
!cat untitled.txt

1234
1111
2222
3333

In [5]:
!xxd untitled.txt

00000000: 3132 3334 0a31 3131 310a 3232 3232 0a33  1234.1111.2222.3
00000010: 3333 33                                  333


Pour la manipulation de bas niveau, nous utiliserons le module `struct` pour encoder (`.pack()`) et décoder(`.unpack()`) nos séquences binaires.

**Syntaxe**:
```python
struct.pack(fmt, v1, v2, ...)
struct.unpack(fmt, buffer)
struct.calcsize(fmt)¶
```

#### Struct (chaîne de formatage)

| Format | Représentation   | Taille              |
|--------|------------------|---------------------|
| `c`    | caractère        | 1 octet                    
| `b`    | entier signé     | 1 octet
| `B`    | entier non-signé |1 octet
| `?`    | booléen          | 1 octet
| `h`    | entier signé     | 2 octets
| `H`    | entier non-signé | 2 octets
| `i`    | entier signé     | 4 octets
| `I`    | entier non-signé | 4 octets
| `l`    | entier signé     | 4 octets
| `L`    | entier non-signé | 4 octets
| `q`    | entier signé     | 8 octets
| `Q`    | entier non-signé | 8 octets
| `f`    | point-flottant<br>simple-précision | 4 octets
| `d`    | point-flottant<br>double-précision | 8 octets
| `s`    | chaîne de caractères<br> | (préciser la taille)

| Format | Ordre des octets | Alignement        |
|--------|------------------|---------------------|
| `@`    | natif            | natif
| `=`    | natif            | aucun
| `<`    | little-endian    | aucun
| `>`    | big-endian       | aucun


#### Exemple 1 

```python
  struct.pack('<hhh', -2,1,258)
```

In [8]:
import struct
print(struct.pack('<hhh', -2,1,258))
print('Taille:', struct.calcsize('<hhh'))

b'\xfe\xff\x01\x00\x02\x01'
Taille: 6


#### Exemple 2 

```python
  struct.pack('>d', 4.5)
```

In [13]:
import struct
octets = struct.pack('>d', 4.5)
for unOctet in octets:
    print("%02x" % unOctet,end=' ')
print('\nTaille:', struct.calcsize('>d'))

40 12 00 00 00 00 00 00 
Taille: 8


Preuve: http://www.binaryconvert.com/result_double.html?decimal=052046053

In [14]:
import struct #P-E à l'examen
print( struct.calcsize('<hhihbbbhi') )

19


## Encoder des chaînes de caractères

Avec le format 's', par exemple '5s', on représente les octets de la chaîne sur 5 octets.  

Considérons d'abord des caractères ASCII 7 bits (1 caractère = 1 octet).

Si on encode `ABCDE`, nous aurons les cinq octets stockés.

In [7]:
import struct
octets = struct.pack('5s', 'ABCDE'.encode('ascii'))
for unOctet in octets:
    print("%x" % unOctet,end=' ')
print('')
for unOctet in octets:
    print(" %c" % unOctet,end=' ')
print('')

41 42 43 44 45 
 A  B  C  D  E 


Qu'arrive-t-il si on a seulement 3 lettres pour un espace de 5 caractères?
Les caractères inutilisés sont annotés en zéro binaire.

In [18]:
import struct
octets = struct.pack('5s', 'ABC'.encode('ascii'))
for unOctet in octets:
    print("%02x" % unOctet,end=' ')
print('')
for unOctet in octets:
    print(" %c" % unOctet,end=' ')
print('')

41 42 43 00 00 
 A  B  C       


S'il y en a plus? Tronqués...

In [19]:
import struct
octets = struct.pack('5s', 'ABCDEFGH'.encode('ascii'))
for unOctet in octets:
    print("%x" % unOctet,end=' ')
print('')
for unOctet in octets:
    print(" %c" % unOctet,end=' ')
print('')

41 42 43 44 45 
 A  B  C  D  E 


### Encodage

Qu'arrive-t-il si on tente d'encoder une lettre accentuée?

In [11]:
'é'.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\xe9' in position 0: ordinal not in range(128)

Normal d'avoir une erreur car ce caractère Unicode est `00C9`.  

Pour l'encodage ISO-8859-1 (latin-1), on pourra le mettre sur un octet.

Pour l'encodage UTF-8, il sera sur deux octets.

In [13]:
print('ISO-8859-1:', len('é'.encode('iso-8859-1')) )
for unOctet in 'é'.encode('iso-8859-1'):
    print("%x" % unOctet,end=' ')
print('')
print('UTF-8     :', len('é'.encode('UTF-8')) )
for unOctet in 'é'.encode('UTF-8'):
    print("%x" % unOctet,end=' ')
print('')

ISO-8859-1: 1
e9 
UTF-8     : 2
c3 a9 


Référence: http://www.ltg.ed.ac.uk/~richard/utf-8.cgi?input=%C3%A9&mode=char

Qu'arrive-t-il avec un caractère cyrillique?

In [15]:
print('UTF-8:', len('Доброе утро'.encode('utf-8')) )
for unOctet in 'Доброе утро'.encode('utf-8'):
    print("%x" % unOctet,end=' ')
print('')

UTF-8: 21
d0 94 d0 be d0 b1 d1 80 d0 be d0 b5 20 d1 83 d1 82 d1 80 d0 be 


Référence: http://www.ltg.ed.ac.uk/~richard/utf-8.cgi?input=%D0%94&mode=char

In [27]:
print('UTF-8:', len('😎'.encode('utf-8')) )
for unOctet in '😎'.encode('utf-8'):
    print("%x" % unOctet,end=' ')
print('')
print('Unicode: %06x' % ord(u'😎'))

UTF-8: 4
f0 9f 98 8e 
Unicode: 01f60e


Référence: http://www.ltg.ed.ac.uk/~richard/utf-8.cgi?input=%F0%9F%98%8E&mode=char